# basic LR model

 - normalising the data

In [20]:
# imports
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score
seed = 12345

In [21]:
training = pd.read_csv('training2.csv')
holdout = pd.read_csv('holdout2.csv')
training.head(5)

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,virksomhedsoplysninger_21,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,virksomhedsoplysninger_20,ejerkreds_aktiv_10,...,ejerkreds_ny_32,ejerkreds_aktiv_34,virksomhedsoplysninger_55,virksomhedsoplysninger_54,sagsoplysninger_3,ejerkreds_ny_34,sagsoplysninger_2,sagsoplysninger_30,sagsoplysninger_36,target
0,1,0,0,0.0,"-5,00e+06","-1,0e+06","2,87250000000000e-02","23,89750000000000",-500000.0,0.0,...,0.0,0.0,4,5,0,0,-79,0,2,False
1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,0.0,6,5,0,0,-30,0,2,False
2,5,0,0,0.0,"-5,00e+06","0,0e+00","0,00000000000000e+00","12,95000000000000",0.0,0.0,...,0.0,0.0,9,10,0,0,-135,0,2,False
3,6,0,0,0.0,"-5,00e+06","-1,0e+06","1,93000000000000e-01","38,81500000000000",0.0,0.0,...,0.0,0.0,4,10,0,0,0,0,2,True
4,7,1,1,0.0,"-5,00e+06","-1,0e+06","4,71583333333333e-01","171,26750000000001",-500000.0,0.0,...,0.0,0.0,6,10,0,0,-21,1,2,False


In [22]:
if "," in "-5,123":
    print("comma found")

comma found


In [23]:
# one or more of the columns with floats/scientific notation has , instead of . so need to convert them
def convert_commas(df):
    '''takes a pandas dataframe as input
       converts all , into .
       returns the converted pandas dataframe'''
    
    for (columnName, columnData) in df.iteritems():
        # check if any of the variables in the column contain commas
        convert = False
        for i in columnData:
            i = str(i)
            if "," in i:
                convert = True

        # if convert is True then we must change one or more values
        converted = list()
        if convert == True:
            for i in columnData:
                i = str(i)
                if "," in i:
                    new_string = i.replace(",", ".")
                    converted.append(new_string)
                else:
                    converted.append(i)

            # deleting old column and adding new
            df.drop(columnName, axis=1, inplace=True)
            df[columnName] = converted
    
    return df

In [24]:
training = convert_commas(training)
holdout = convert_commas(holdout)

In [25]:
training.head(5)

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,virksomhedsoplysninger_21,virksomhedsoplysninger_20,ejerkreds_aktiv_10,ejerkreds_aktiv_31,ejerkreds_aktiv_96,ejerkreds_aktiv_47,ejerkreds_aktiv_12,...,ejerkreds_ny_34,sagsoplysninger_2,sagsoplysninger_30,sagsoplysninger_36,target,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,ejerkreds_aktiv_25
0,1,0,0,0.0,-500000.0,0.0,4.0,4.0,4,3,...,0,-79,0,2,False,-5.00e+06,-1.0e+06,2.87250000000000e-02,23.89750000000000,-25307.0000000000
1,2,0,0,NaN,NaN,0.0,2.0,1.0,4,1,...,0,-30,0,2,False,nan,nan,nan,nan,-25537.0000000000
2,5,0,0,0.0,0.0,0.0,4.0,1.0,6,1,...,0,-135,0,2,False,-5.00e+06,0.0e+00,0.00000000000000e+00,12.95000000000000,-25432.0000000000
3,6,0,0,0.0,0.0,0.0,3.0,1.0,4,2,...,0,0,0,2,True,-5.00e+06,-1.0e+06,1.93000000000000e-01,38.81500000000000,-25564.0000000000
4,7,1,1,0.0,-500000.0,0.0,13.0,3.0,20,1,...,0,-21,1,2,False,-5.00e+06,-1.0e+06,4.71583333333333e-01,171.26750000000001,-25515.0000000000


In [26]:
import random
# need to replace nan values in both training and holdout datasets

# if values are binary, replace them randomly with 0 or 1 with more likelihood based on the mean

# if values are continuous, replace them with the mean

# if values are categorical, replace them randomly with the categories based on the likelihood of each one

confirmed_categorical_list = ['ejerkreds_aktiv_130', 'ejerkreds_aktiv_129', 'ejerkreds_aktiv_127',
                                      'ejerkreds_aktiv_80', 'ejerkreds_aktiv_102', 'ejerkreds_ny_32', 'ejerkreds_aktiv_34']

for (columnName, columnData) in training.iteritems():
    # checking if values are binary, continuous, or categorical
    unique_values = list()
    for value in columnData:
        value = str(value)
        if value not in unique_values:
            unique_values.append(value)
            
    #print(unique_values)
    
    # if nan is not in the column do nothing
    if "nan" in unique_values:
        
        # if it is a binary column with nan (there is only 1 in this case)
        if len(unique_values) == 3:
            print(unique_values)

            # calculating the percentage that are 1
            total = training.shape[0]
            one_count = 0
            for i in columnData:
                i = str(i)
                if i == "1.0":
                    one_count += 1

            one_percent = one_count/total
            #print(one_percent)
            # rounding the percent to create a distribution list
            rounded = round(one_percent)
            #print(rounded)

            num_ones = rounded
            num_zeros = 100-num_ones
            one_list = [1] * num_ones
            zero_list = [0] * num_zeros

            distribution = zero_list + one_list
            #print(distribution)

            new_values = list()
            for value in columnData:
                value = str(value)
                if value == "0.0":
                    new_values.append(value)
                elif value == "1.0":
                    new_values.append(value)
                elif value == "nan":
                    # pick randomly from the list
                    new_values.append(random.choice(distribution))

            # deleting old column and replacing with new
            training.drop(columnName, axis=1, inplace=True)
            training[columnName] = new_values
            
            # doing the same to holdout
            new_values = list()
            for idx, row in holdout.iterrows():
                value = str(row[columnName])
                
                if value == "0.0":
                    new_values.append(value)
                elif value == "1.0":
                    new_values.append(value)
                elif value == "nan":
                    # pick randomly from list
                    new_values.append(random.choice(distribution))
            holdout.drop(columnName, axis=1, inplace=True)
            holdout[columnName] = new_values

        # if values are categorical
        elif columnName in confirmed_categorical_list:
            to_pick_from = []
            for i in columnData:
                i = str(i)
                if i != "nan":
                    to_pick_from.append(i)

            new_list = []
            for i in columnData:
                i = str(i)
                if i == 'nan':
                    new_list.append(random.choice(to_pick_from))
                else:
                    new_list.append(i)

            # deleting old column and replacing with new
            training.drop(columnName, axis=1, inplace=True)
            training[columnName] = new_list
            
            new_list = []
            for idx, row in holdout.iterrows():
                value = str(row[columnName])
                if value == 'nan':
                    new_list.append(random.choice(to_pick_from))
                else:
                    new_list.append(value)
            
            holdout.drop(columnName, axis=1, inplace=True)
            holdout[columnName] = new_list

        # else the column is continuous, so take the mean and add it
        else:
            non_nan_values = []
            for i in columnData:
                i = str(i)
                if i != "nan":
                    non_nan_values.append(float(i))

            mean = sum(non_nan_values) / len(non_nan_values)

            new_list = []
            for i in columnData:
                i = str(i)
                if i == 'nan':
                    new_list.append(mean)
                else:
                    new_list.append(i)

            # deleting old column and replacing with new
            training.drop(columnName, axis=1, inplace=True)
            training[columnName] = new_list
            
            new_list = []
            for idx, row in holdout.iterrows():
                value = str(row[columnName])
                if value == 'nan':
                    new_list.append(mean)
                else:
                    new_list.append(value)
            
            holdout.drop(columnName, axis=1, inplace=True)
            holdout[columnName] = new_list

['0.0', 'nan', '1.0']


In [27]:
training.head()

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,ejerkreds_aktiv_47,ejerkreds_aktiv_12,ejerkreds_aktiv_48,ejerkreds_aktiv_50,ejerkreds_aktiv_49,ejerkreds_aktiv_78,virksomhedsoplysninger_55,...,ejerkreds_aktiv_26,ejerkreds_aktiv_80,ejerkreds_aktiv_102,ejerkreds_ny_32,ejerkreds_aktiv_34,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,ejerkreds_aktiv_25
0,1,0,0,4,3,1,1,1,0,4,...,-25307.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,2.87250000000000e-02,23.89750000000000,-25307.0000000000
1,2,0,0,4,1,1,2,1,0,6,...,-25537.0,0.0,0.0,0.0,0.0,-6627466.525722,-1404997.351227,0.295905,110.517668,-25537.0000000000
2,5,0,0,6,1,4,4,4,0,9,...,-25432.0,0.0,0.0,0.0,0.0,-5.00e+06,0.0e+00,0.00000000000000e+00,12.95000000000000,-25432.0000000000
3,6,0,0,4,2,2,2,2,0,4,...,-25564.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,1.93000000000000e-01,38.81500000000000,-25564.0000000000
4,7,1,1,20,1,2,5,2,0,6,...,-25515.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,4.71583333333333e-01,171.26750000000001,-25515.0000000000


In [28]:
holdout.head()

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,ejerkreds_aktiv_47,ejerkreds_aktiv_12,ejerkreds_aktiv_48,ejerkreds_aktiv_50,ejerkreds_aktiv_49,ejerkreds_aktiv_78,virksomhedsoplysninger_55,...,ejerkreds_aktiv_26,ejerkreds_aktiv_80,ejerkreds_aktiv_102,ejerkreds_ny_32,ejerkreds_aktiv_34,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,ejerkreds_aktiv_25
0,3538,0,0,7,1,4,5,3,0,9,...,-25561.0,0.0,0.0,0.0,0.0,-5.00e+06,0.0e+00,0.00000000000000e+00,14.10000000000000,-25561.0000000000
1,12138,0,0,1,1,1,1,1,1,4,...,-25559.0,0.0,0.0,0.0,1.0,-6627466.525722,-1404997.351227,0.295905,110.517668,-25559.0000000000
2,9794,1,1,18,2,4,4,4,0,4,...,-25566.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,2.22312500000000e-01,159.74375000000001,-25566.0000000000
3,12966,1,1,6,1,2,2,2,0,7,...,-25565.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,4.00000000000000e-01,203.01800000000000,-25565.0000000000
4,8724,1,0,4,1,2,2,2,0,5,...,-25567.0,0.0,0.0,0.0,0.0,0.00e+00,0.0e+00,0.00000000000000e+00,1339.00000000000000,-25567.0000000000


In [29]:
training['target'].value_counts()

False    10335
True      3384
Name: target, dtype: int64

In [30]:
# resampling to balance true and false frauds
from sklearn.utils import resample

# Separate majority and minority classes
training_majority = training[training.target==False]
training_minority = training[training.target==True]

# Upsample minority class
training_minority_upsampled = resample(training_minority, 
                                 replace=True,            # sample with replacement
                                 n_samples=10335,         # to match majority class
                                 random_state=seed)       # reproducible results

# Combine majority class with upsampled minority class
training_upsampled = pd.concat([training_majority, training_minority_upsampled])

# Display new class counts
training_upsampled.target.value_counts()

False    10335
True     10335
Name: target, dtype: int64

In [31]:
# train test splitting
from sklearn.model_selection import train_test_split

X = training_upsampled.drop(['target', 'virksomhedsRegistreringsId'], axis=1)
y = training_upsampled['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=seed)

In [32]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier().fit(X_train, y_train)
preds = clf.predict(X_val)

print(accuracy_score(y_val, preds))

0.7250073292289652


In [33]:
holdout.head()

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,ejerkreds_aktiv_47,ejerkreds_aktiv_12,ejerkreds_aktiv_48,ejerkreds_aktiv_50,ejerkreds_aktiv_49,ejerkreds_aktiv_78,virksomhedsoplysninger_55,...,ejerkreds_aktiv_26,ejerkreds_aktiv_80,ejerkreds_aktiv_102,ejerkreds_ny_32,ejerkreds_aktiv_34,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,ejerkreds_aktiv_25
0,3538,0,0,7,1,4,5,3,0,9,...,-25561.0,0.0,0.0,0.0,0.0,-5.00e+06,0.0e+00,0.00000000000000e+00,14.10000000000000,-25561.0000000000
1,12138,0,0,1,1,1,1,1,1,4,...,-25559.0,0.0,0.0,0.0,1.0,-6627466.525722,-1404997.351227,0.295905,110.517668,-25559.0000000000
2,9794,1,1,18,2,4,4,4,0,4,...,-25566.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,2.22312500000000e-01,159.74375000000001,-25566.0000000000
3,12966,1,1,6,1,2,2,2,0,7,...,-25565.0,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,4.00000000000000e-01,203.01800000000000,-25565.0000000000
4,8724,1,0,4,1,2,2,2,0,5,...,-25567.0,0.0,0.0,0.0,0.0,0.00e+00,0.0e+00,0.00000000000000e+00,1339.00000000000000,-25567.0000000000


In [34]:
# there are still two nans in sagsoplysninger_2, it is continuous so let's take the mean in the test set and throw
# it on the nans

float_list = []
for idx, row in holdout.iterrows():
    value = str(row['sagsoplysninger_2'])
    if value != 'nan':
        float_list.append(float(value))
    
# get average of list
mean = sum(float_list) / len(float_list)

# for all nans replace them with the mean
new_list = []
for idx, row in holdout.iterrows():
    value = str(row['sagsoplysninger_2'])
    if value != 'nan':
        new_list.append(value)
    elif value == 'nan':
        new_list.append(mean)

holdout.drop('sagsoplysninger_2', axis=1, inplace=True)
holdout['sagsoplysninger_2'] = new_list

In [35]:
holdout.head()

,virksomhedsRegistreringsId,ejerkreds_aktiv_92,ejerkreds_aktiv_94,ejerkreds_aktiv_47,ejerkreds_aktiv_12,ejerkreds_aktiv_48,ejerkreds_aktiv_50,ejerkreds_aktiv_49,ejerkreds_aktiv_78,virksomhedsoplysninger_55,...,ejerkreds_aktiv_80,ejerkreds_aktiv_102,ejerkreds_ny_32,ejerkreds_aktiv_34,virksomhedsoplysninger_22,virksomhedsoplysninger_25,virksomhedsoplysninger_24,virksomhedsoplysninger_23,ejerkreds_aktiv_25,sagsoplysninger_2
0,3538,0,0,7,1,4,5,3,0,9,...,0.0,0.0,0.0,0.0,-5.00e+06,0.0e+00,0.00000000000000e+00,14.10000000000000,-25561.0000000000,0.0
1,12138,0,0,1,1,1,1,1,1,4,...,0.0,0.0,0.0,1.0,-6627466.525722,-1404997.351227,0.295905,110.517668,-25559.0000000000,-8.0
2,9794,1,1,18,2,4,4,4,0,4,...,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,2.22312500000000e-01,159.74375000000001,-25566.0000000000,-1.0
3,12966,1,1,6,1,2,2,2,0,7,...,0.0,0.0,0.0,0.0,-5.00e+06,-1.0e+06,4.00000000000000e-01,203.01800000000000,-25565.0000000000,-2.0
4,8724,1,0,4,1,2,2,2,0,5,...,0.0,0.0,0.0,0.0,0.00e+00,0.0e+00,0.00000000000000e+00,1339.00000000000000,-25567.0000000000,0.0


In [36]:
# testing base model predictions
X_test = holdout.drop(['virksomhedsRegistreringsId'], axis=1)

# ordering X_test columns to be the same as X_train
X_test = X_test[X_train.columns]

preds = clf.predict(X_test)
probs = clf.predict_proba(X_test)
print(preds)
print(probs)

# print(holdout['virksomhedsRegistreringsId'])

# changing preds to be 0's and 1's instead of True and False
preds_converted = []
for i in preds:
    if i == True:
        preds_converted.append(1)
    elif i == False:
        preds_converted.append(0)
        
# changing probs to be only the higher probability instead of both
probs_converted = []
for i in probs:
    probs_converted.append(i[1])

# mapping the preds to the virksomheds
data_tuples = list(zip(holdout['virksomhedsRegistreringsId'].values, probs_converted))
df_out = pd.DataFrame(data_tuples, columns=['virksomhedsRegistreringsId','targetProbability'])
df_out['targetPredicted'] = preds_converted

df_out.head()

[ True False  True ... False False  True]
[[0.4680825  0.5319175 ]
 [0.70276059 0.29723941]
 [0.32594506 0.67405494]
 ...
 [0.50636608 0.49363392]
 [0.85547991 0.14452009]
 [0.3624677  0.6375323 ]]


,virksomhedsRegistreringsId,targetProbability,targetPredicted
0,3538,0.531917,1
1,12138,0.297239,0
2,9794,0.674055,1
3,12966,0.756421,1
4,8724,0.539500,1


In [37]:
# saving prediction for upload
df_out.to_csv('forth_upload.csv', index=False, sep=';')

In [38]:
true_count = 0
false_count = 0

for i in preds:
    if i == True:
        true_count += 1
    elif i == False:
        false_count += 1

print(true_count)
print(false_count)

1345
2084
